## IMPORT PACKAGES

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

## DATA PREPROCESSING

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True) #2-tuple structure [input, target]

In [3]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples #take 10% of test data as validation set
num_validation_samples = tf.cast(num_validation_samples, tf.int64) #convert to integer
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255. #converted pixel value as to be between 0 and 1, dot:floating-point
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale) #scale whole data set
test_data = mnist_test.map(scale)

#shuffling: keeping the same information in different order
BUFFER_SIZE = 10000 #prevents memory leak for large data sets
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples) #skip validation data

BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

## OUTLINE THE MODEL

In [4]:
input_size = 784
output_size = 10
hidden_layer_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
                            ])

## OPTIMIZER and LOSS FUNCTION

In [5]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## TRAINING

In [6]:
NUM_EPOCHS = 5
model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), validation_steps=10, verbose=2)

Epoch 1/5
540/540 - 5s - loss: 0.4140 - accuracy: 0.8838 - val_loss: 0.2187 - val_accuracy: 0.9365
Epoch 2/5
540/540 - 4s - loss: 0.1808 - accuracy: 0.9472 - val_loss: 0.1538 - val_accuracy: 0.9538
Epoch 3/5
540/540 - 4s - loss: 0.1360 - accuracy: 0.9605 - val_loss: 0.1337 - val_accuracy: 0.9615
Epoch 4/5
540/540 - 4s - loss: 0.1139 - accuracy: 0.9663 - val_loss: 0.1145 - val_accuracy: 0.9663
Epoch 5/5
540/540 - 4s - loss: 0.0968 - accuracy: 0.9711 - val_loss: 0.1046 - val_accuracy: 0.9702


## TESTING

In [7]:
test_loss, test_accuracy = model.evaluate(test_data)

      1/Unknown - 1s 934ms/step - loss: 0.1218 - accuracy: 0.9627

In [8]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.12. Test accuracy: 96.27%
